In [0]:
import numpy as np

batch = 64
emb_dim = 1024

np.random.seed(1234)
emb1 = np.random.rand(batch,emb_dim)
np.random.seed(2345)
emb2 = np.random.rand(batch,emb_dim)
margin = 0.3
labels = np.expand_dims(np.concatenate((np.arange(batch/2),np.arange(batch/2)), axis=0), axis=1)

In [0]:
def _distance_metric(x, y):
    """
    Args:
        x: tensor, with shape [m, d], (batch_size, d)
        y: tensor, with shape [n, d], (batch_size, d)
    Returns:
        dist: tensor, with shape [m, n], (batch_size, batch_size)
    """
    # |x-y|^2 = x^2 - 2xy + y^2
    # xy
    xy = np.matmul(x, np.transpose(y))
    # x^2
    xx = np.matmul(x, np.transpose(x))
    xx = np.diag(xx)
    # y^2
    yy = np.matmul(y, np.transpose(y))
    yy = np.diag(yy)
    '''
    (batch_size,1)-(batch_size,batch_size):
        Equivalent to each column operation
    (batch_size,batch_size)+(1,batch_size):
        Equivalent to each row operation
    '''
    distances = np.expand_dims(xx, 1) - 2.0*xy + np.expand_dims(yy, 0)
    return distances

In [0]:
def _label_mask(labels):
    '''
    if label is same, label_mask will return True
    ------------------------------------
    Args:
        labels:     Label Data, shape = (batch_size,1)
    Returns:
        label_mask: tensor, with shape [m, n], (batch_size, batch_size)
        ex.
            labels = [1,0,1]
            label_mask = [[1, 0, 1],
                          [0, 1, 0],
                          [1, 0, 1]]
    '''
    label_mask = _distance_metric(labels, labels)
    label_mask = (label_mask == 0).astype(np.float32)
    return label_mask

In [0]:
def batch_all(labels, emb1, emb2, margin):
    '''
    batch all triplet loss of a batch
    ------------------------------------
    Args:
        labels:     Label Data, shape = (batch_size,1)
        emb1, emb2: Embedding Feature, shape = (batch_size, vector_size)
        margin:     margin, scalar
    Returns:
        triplet_loss: scalar, for one batch
    '''
    dist_mat = _distance_metric(emb1, emb2)
    # an and ap mask
    ap_mask = _label_mask(labels)
    an_mask = np.logical_not(ap_mask).astype(np.float32)
    # distance between anchor and positive
    dist_ap = np.sum(dist_mat*ap_mask, axis=1)/np.sum(ap_mask, axis=1)
    # ap - dist_mat + margin
    mat = np.expand_dims(dist_ap, 1) - dist_mat + margin
    # only need ap-an
    mat = mat*an_mask
    # caluculate the number of valid triplet loss
    mask = np.greater(mat, margin).astype(np.float32)
    num_valid_triplets = np.sum(mask)
    triplet_loss = mat*mask
    # <1 : 1
    num_valid_triplets = np.maximum(num_valid_triplets, 1.0)
    # divided triplet_loss by num_valid_triplets
    triplet_loss = np.sum(triplet_loss)/num_valid_triplets
    return triplet_loss, num_valid_triplets

In [0]:
triplet_loss, num_valid_triplets = batch_all(labels, emb1, emb2, margin)

In [6]:
triplet_loss

5.893566522818261

In [7]:
num_valid_triplets

1876.0

In [0]:
def batch_hard(labels, emb1, emb2, margin):
    '''
    batch all triplet loss of a batch
    ------------------------------------
    Args:
        labels:     Label Data, shape = (batch_size,1)
        emb1, emb2: Embedding Feature, shape = (batch_size, vector_size)
        margin:     margin, scalar
    Returns:
        triplet_loss: scalar, for one batch
    '''
    dist_mat = _distance_metric(emb1, emb2)
    # an and ap mask
    ap_mask = _label_mask(labels)
    an_mask = np.logical_not(ap_mask).astype(np.float32)
    # distance between anchor and positive
    dist_ap = np.sum(dist_mat*ap_mask, axis=1)/np.sum(ap_mask, axis=1)
    # caluculate the number of valid triplet loss
    mat = np.expand_dims(dist_ap, 1) - dist_mat + margin
    mat = mat*an_mask
    mask = np.greater(mat, margin).astype(np.float32)
    num_valid_triplets = np.sum(mask)
    
    # the max of distance between anchor and positive
    dist_ap = np.max(dist_mat*ap_mask, axis=1)
    # the max of distance between anchor and negative
    max_num = np.max(dist_mat)
    dist_an = dist_mat*an_mask + ap_mask*max_num
    dist_an = np.amin(dist_an, axis=1)
    # ap - dist_mat + margin
    mat = dist_ap - dist_an + margin
    triplet_loss = np.mean(mat)
    # <1 : 1
    triplet_loss = np.maximum(triplet_loss, 1e-16)
    return triplet_loss, num_valid_triplets

In [0]:
triplet_loss, num_valid_triplets = batch_hard(labels, emb1, emb2, margin)

In [10]:
triplet_loss

16.3194424177763

In [11]:
num_valid_triplets

1876.0